In [1]:
import tensorflow as tf
tf.executing_eagerly()

True

In [47]:
#Used in Tensorflow Model
import numpy as np
import tflearn
import random
import pandas as pd

#Used to for Contextualisation and Other NLP Tasks.
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

#Other
import json
import pickle
import warnings
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
warnings.filterwarnings("ignore")
print(tf.__version__)

2.6.0-dev20210528


In [3]:
print("Processing the Intents.....")
with open('intents.json') as json_data:
    intents = json.load(json_data)

Processing the Intents.....


In [4]:
words = []
classes = []
documents = []
ignore_words = ['?']
print("Looping through the Intents to Convert them to words, classes, documents and ignore_words.......")
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

Looping through the Intents to Convert them to words, classes, documents and ignore_words.......


In [5]:
print("Stemming, Lowering and Removing Duplicates.......")
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

Stemming, Lowering and Removing Duplicates.......
51 documents
8 classes ['goodbye', 'greeting', 'jokes', 'kosong', 'moodbaik', 'moodburuk', 'perkenalan', 'thanks']
86 unique stemmed words ['!', ',', 'ad', 'aj', 'akhirny', 'aku', 'antusia', 'ap', 'apakah', 'bahag', 'banget', 'banyak', 'bersemang', 'bersyuk', 'bis', 'boleh', 'bro', 'capek', 'cer', 'dadah', 'deng', 'dengerin', 'dir', 'dulu', 'feel', 'ga', 'galau', 'good', 'hai', 'hal', 'halo', 'har', 'hello', 'hey', 'hi', 'in', 'ingin', 'itu', 'iâ€™m', 'jad', 'jok', 'jug', 'jump', 'kab', 'kamu', 'kasih', 'kegy', 'ken', 'kesel', 'kurang', 'lag', 'lelucon', 'lib', 'lucu', 'makasih', 'mau', 'membantu', 'menyelesaik', 'nam', 'nang', 'nyerah', 'obrol', 'orang', 'overthink', 'percay', 'perkenalk', 'rasany', 'sampa', 'sang', 'selesa', 'semang', 'senang', 'siap', 'stress', 'sudah', 'terim', 'terimakasih', 'terserah', 'thank', 'tid', 'tugasku', 'up', 'what', 'ya', 'yang', 'ye']


In [6]:
print("Creating the Data for our Model.....")
training = []
output = []
print("Creating an List (Empty) for Output.....")
output_empty = [0] * len(classes)

print("Creating Traning Set, Bag of Words for our Model....")
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

Creating the Data for our Model.....
Creating an List (Empty) for Output.....
Creating Traning Set, Bag of Words for our Model....


In [7]:
print("Shuffling Randomly and Converting into Numpy Array for Faster Processing......")
random.shuffle(training)
training = np.array(training)

print("Creating Train and Test Lists.....")
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Building Neural Network for Out Chatbot to be Contextual....")
print("Resetting graph data....")


Shuffling Randomly and Converting into Numpy Array for Faster Processing......
Creating Train and Test Lists.....
Building Neural Network for Out Chatbot to be Contextual....
Resetting graph data....


In [8]:
# %tensorflow_version 1.x
tf.compat.v1.reset_default_graph()

In [9]:
# model versi keras
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(units=128, input_shape=(len(train_x[0]),), activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units=len(train_y[0]), activation='softmax')
])

In [10]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
hist = model.fit(tf.constant(train_x), tf.constant(train_y),
                 epochs=100, batch_size=32, verbose=1, 
                steps_per_epoch=20)
model.save('chatbot_model.h5', hist)
model.save('model.tflearn', hist)
print("model created")

Train on 20 samples
Epoch 1/100
20/20 [==============================] - 0s 4ms/step - batch: 9.5000 - size: 1.0000 - loss: 2.0697 - acc: 0.1539
Epoch 2/100
20/20 [==============================] - 0s 4ms/step - batch: 9.5000 - size: 1.0000 - loss: 1.9986 - acc: 0.2088
Epoch 3/100
20/20 [==============================] - 0s 4ms/step - batch: 9.5000 - size: 1.0000 - loss: 1.9550 - acc: 0.2225
Epoch 4/100
20/20 [==============================] - 0s 4ms/step - batch: 9.5000 - size: 1.0000 - loss: 1.9138 - acc: 0.2461
Epoch 5/100
20/20 [==============================] - 0s 4ms/step - batch: 9.5000 - size: 1.0000 - loss: 1.8490 - acc: 0.2931
Epoch 6/100
20/20 [==============================] - 0s 4ms/step - batch: 9.5000 - size: 1.0000 - loss: 1.7708 - acc: 0.3294
Epoch 7/100
20/20 [==============================] - 0s 5ms/step - batch: 9.5000 - size: 1.0000 - loss: 1.6789 - acc: 0.3667
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - batch: 9.5000 - size: 1.0000 - loss: 1

20/20 [==============================] - 0s 4ms/step - batch: 9.5000 - size: 1.0000 - loss: 0.1285 - acc: 0.9667
Epoch 64/100
20/20 [==============================] - 0s 4ms/step - batch: 9.5000 - size: 1.0000 - loss: 0.1258 - acc: 0.9676
Epoch 65/100
20/20 [==============================] - 0s 4ms/step - batch: 9.5000 - size: 1.0000 - loss: 0.1211 - acc: 0.9598
Epoch 66/100
20/20 [==============================] - 0s 4ms/step - batch: 9.5000 - size: 1.0000 - loss: 0.1091 - acc: 0.9745
Epoch 67/100
20/20 [==============================] - 0s 5ms/step - batch: 9.5000 - size: 1.0000 - loss: 0.1131 - acc: 0.9686
Epoch 68/100
20/20 [==============================] - 0s 5ms/step - batch: 9.5000 - size: 1.0000 - loss: 0.1299 - acc: 0.9657
Epoch 69/100
20/20 [==============================] - 0s 4ms/step - batch: 9.5000 - size: 1.0000 - loss: 0.1122 - acc: 0.9716
Epoch 70/100
20/20 [==============================] - 0s 4ms/step - batch: 9.5000 - size: 1.0000 - loss: 0.1052 - acc: 0.9716
Epoch

In [67]:
import pathlib
tf.saved_model.save(model, '/tmp/saved_model/1/') #will be saved in dick C


# import tensorflow.keras.backend
# # Convert the model.
# converter = tf.lite.TFLiteConverter.from_keras_model(model)
# tflite_model = converter.convert()
# model.save('/tmp/my_model', hist)
converter = tf.lite.TFLiteConverter.from_saved_model('/tmp/saved_model/1/') # path to the SavedModel directory
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# # Save the model.
# with open('model.tflite', 'wb') as f:
#     f.write(tflite_model)

tflite_model_file = pathlib.Path('/tmp/model.tflite')
tflite_model_file.write_bytes(tflite_model)

    


FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadata field will be deprecated soon, so please move the metadata to a different file.



FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadata field will be deprecated soon, so please move the metadata to a different file.


INFO:tensorflow:Assets written to: /tmp/saved_model/1/assets


INFO:tensorflow:Assets written to: /tmp/saved_model/1/assets


INFO:tensorflow:Restoring parameters from /tmp/saved_model/1/variables\variables


...

INFO:tensorflow: tensor name: StatefulPartitionedCall:0, shape: (-1, 8), type: DT_FLOAT


INFO:tensorflow: tensor name: StatefulPartitionedCall:0, shape: (-1, 8), type: DT_FLOAT


INFO:tensorflow:Restoring parameters from /tmp/saved_model/1/variables\variables


INFO:tensorflow:Restoring parameters from /tmp/saved_model/1/variables\variables
Use '@tf.function' or '@defun' to decorate the function.
Use '@tf.function' or '@defun' to decorate the function.
Use '@tf.function' or '@defun' to decorate the function.
Use '@tf.function' or '@defun' to decorate the function.
Use '@tf.function' or '@defun' to decorate the function.
Use '@tf.function' or '@defun' to decorate the function.
Use '@tf.function' or '@defun' to decorate the function.
Use '@tf.function' or '@defun' to decorate the function.
Use '@tf.function' or '@defun' to decorate the function.
Use '@tf.function' or '@defun' to decorate the function.
Use '@tf.function' or '@defun' to decorate the function.
Use '@tf.function' or '@defun' to decorate the function.
Use '@tf.function' or '@defun' to decorate the function.
Use '@tf.function' or '@defun' to decorate the function.
Use '@tf.function' or '@defun' to decorate the function.
Use '@tf.function' or '@defun' to decorate the function.
Use '@t

26608

In [68]:
print("Pickle is also Saved..........")
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

Pickle is also Saved..........


In [135]:
print("Loading Pickle.....")
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']


with open('intents.json') as json_data:
    intents = json.load(json_data)
    
# load our saved model
# model.load('./model.tflearn')

# # Recreate the exact same model, including its weights and the optimizer
# new_model = tf.keras.models.load_model('chatbot_model.h5')

# # Show the model architecture
# new_model.summary()

interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()[0]['index']
output_details = interpreter.get_output_details()[0]['index']

Loading Pickle.....


In [136]:
def clean_up_sentence(sentence):
    # It Tokenize or Break it into the constituents parts of Sentense.
    sentence_words = nltk.word_tokenize(sentence)
    # Stemming means to find the root of the word.
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

In [137]:
# Return the Array of Bag of Words: True or False and 0 or 1 for each word of bag that exists in the Sentence
def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

ERROR_THRESHOLD = 0.25
print("ERROR_THRESHOLD = 0.25")

ERROR_THRESHOLD = 0.25


In [138]:
# p = bow("whats", words)
# print (p)
# print (classes)

# Print input shape and type
print(interpreter.get_input_details()[0]['shape'])  # Example: [1 86]
print(interpreter.get_input_details()[0]['dtype'])  # Example: <class 'numpy.float32'>

# Print output shape and type
print(interpreter.get_output_details()[0]['shape'])  # Example: [1 8]
print(interpreter.get_output_details()[0]['dtype'])  # Example: <class 'numpy.float32'>

[ 1 86]
<class 'numpy.float32'>
[1 8]
<class 'numpy.float32'>


In [141]:
def classify(sentence):
#     # generate probabilities from the model
    input_data = pd.DataFrame([bow(sentence, words)], dtype=np.float32, index=['input'])
#     # Prediction or To Get the Posibility or Probability from the Model
#     results = new_model.predict([input_data])[0]
    interpreter.set_tensor(input_details, ([input_data])[0])
    interpreter.invoke()
    results = interpreter.get_tensor(output_details)[0]
#     print(results)
    # Exclude those results which are Below Threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # Sorting is Done because heigher Confidence Answer comes first.
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1])) #Tuppl -> Intent and Probability
    return return_list

classify('hi')

[9.3528411e-07 9.9923193e-01 4.8578670e-07 4.1155235e-04 6.6970861e-06
 3.2506164e-04 2.6503088e-08 2.3289105e-05]


[('greeting', 0.99923193)]

In [142]:
def response(sentence, show_details=False):
    results = classify(sentence)
    # That Means if Classification is Done then Find the Matching Tag.
    if results:
        # Long Loop to get the Result.
        while results:
            for i in intents['intents']:
                # Tag Finding
                if i['tag'] == results[0][0]:
                    # Random Response from High Order Probabilities
                    return print(random.choice(i['responses']))

            results.pop(0)

In [143]:
from time import sleep
print("Hai, hari yang indah")
sleep(0.5)
print("Perkenalkan, saya Zassy. Chatbot pribadi anda")
sleep(0.5)
print("Ada yang bisa saya bantu?")
while True:
    input_data = input("You : ")
    if input_data.lower()== "quit" :
        break
    answer = response(input_data)
    answer

Hai, hari yang indah
Perkenalkan, saya Zassy. Chatbot pribadi anda
Ada yang bisa saya bantu?
You : ho
[5.8034640e-02 3.2779753e-02 1.5675671e-02 8.5620922e-01 2.4219338e-02
 2.7972863e-03 4.1357061e-04 9.8705115e-03]
Pesan kamu sepertinya kosong
You : hi
[9.3528411e-07 9.9923193e-01 4.8578670e-07 4.1155235e-04 6.6970861e-06
 3.2506164e-04 2.6503088e-08 2.3289105e-05]
Hai juga, Bagaimana dengan hari ini?
You : quit
